In [6]:
import pyupbit
import time
import datetime
from pyupbit import get_current_price, get_tickers, get_ohlcv
# , get_ohlcv, get_daily_ohlcv_from_base
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
pattern = [100, 90, 80, 70, 90, 120, 140]
tickers = get_tickers(fiat='KRW')

dfs = []
for ticker in tickers:
    df = get_ohlcv(ticker, 7)
    dfs.append(df['close'])
    time.sleep(0.2)
    
df = pd.concat(dfs, axis=1)
df.columns = tickers
df['target'] = pattern
corr = df.corr()

corr.loc['target'].sort_values()

KRW-BCH     -0.834684
KRW-WAVES   -0.755797
KRW-ADA     -0.504082
KRW-ALGO    -0.407463
KRW-DOGE    -0.395247
               ...   
KRW-SOL      0.656336
KRW-XTZ      0.706898
KRW-KNC      0.736655
KRW-MANA     0.762601
target       1.000000
Name: target, Length: 111, dtype: float64

In [12]:
df = get_ohlcv('KRW-BTC', interval='minute1', to='20220201 10:00:00')
df

,open,high,low,close,volume,value
2022-02-01 06:40:00,47294000.0,47325000.0,47293000.0,47293000.0,0.512383,2.423516e+07
2022-02-01 06:41:00,47316000.0,47320000.0,47291000.0,47291000.0,0.741421,3.506623e+07
2022-02-01 06:42:00,47291000.0,47315000.0,47291000.0,47293000.0,0.307990,1.457041e+07
2022-02-01 06:43:00,47293000.0,47314000.0,47292000.0,47292000.0,0.266392,1.260030e+07
2022-02-01 06:44:00,47291000.0,47291000.0,47268000.0,47268000.0,0.238327,1.126720e+07
...,...,...,...,...,...,...
2022-02-01 09:55:00,47165000.0,47170000.0,47146000.0,47165000.0,1.089219,5.137480e+07
2022-02-01 09:56:00,47165000.0,47169000.0,47148000.0,47149000.0,1.501486,7.080119e+07
2022-02-01 09:57:00,47149000.0,47151000.0,47146000.0,47148000.0,1.279712,6.033537e+07
2022-02-01 09:58:00,47146000.0,47151000.0,47146000.0,47151000.0,1.829265,8.624722e+07


In [26]:
# 매수 일자 판별
cond = df['high'] >= df['open'] * 1.001
df.index[cond]

DatetimeIndex(['2022-02-01 07:02:00', '2022-02-01 07:30:00',
               '2022-02-01 07:32:00', '2022-02-01 07:40:00',
               '2022-02-01 08:50:00', '2022-02-01 08:52:00',
               '2022-02-01 08:54:00', '2022-02-01 09:03:00',
               '2022-02-01 09:07:00', '2022-02-01 09:08:00',
               '2022-02-01 09:09:00', '2022-02-01 09:22:00',
               '2022-02-01 09:23:00', '2022-02-01 09:24:00',
               '2022-02-01 09:48:00'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
# 매도 조건 탐색 및 수익률 계산
